In [118]:
!pip3 install ipykernel
!pip3 install pandas
import pandas as pd
import numpy as np

In [119]:
def get_data(f, cid):
    data = pd.read_csv(f)
    # data.columns = data.iloc[0]
    # data = data.drop(0)
    #print(data)
    data = data[data['customer_id'] == cid]

    return data

In [120]:
CUSTOMER_ID = 4522
balance_df = get_data("./SampleCompanyData/balance_sheet_view_data.csv", CUSTOMER_ID)
profit_df = get_data("./SampleCompanyData/profit_loss_view_data.csv", CUSTOMER_ID)

In [121]:
balance_df.head()

,id,customer_id,account_id,account_name,period_date,balance,report_id,modified_date
0,43,4522,48,Assets|Fixed Assets|Accumulated Depreciation,2020-11-30,-522042.08,22408,2020-12-13 09:12:00
1,44,4522,49,Assets|Fixed Assets|Accumulated Amortization,2020-11-30,-1333.00,22408,2020-12-13 09:12:00
2,45,4522,51,Assets|Current Assets|Bank Accounts|Savings,2020-11-30,253306.31,22408,2020-12-13 09:12:00
3,46,4522,56,Liabilities|Current Liabilities|Other Current ...,2020-11-30,285931.18,22408,2020-12-13 09:12:00
4,47,4522,62,Equity|Current Year Earnings,2020-11-30,1065940.55,22408,2020-12-13 09:12:00


In [122]:
profit_df.head()

,id,customer_id,account_id,account_name,period_date,balance,report_id,modified_date
0,109,4522,84,Other Income|Wellness Program,2020-11-30,-318.45,24754,2020-12-13 09:13:00
1,110,4522,89,Other Income|Interest Paid,2020-11-30,-2055.48,24754,2020-12-13 09:13:00
2,111,4522,92,Other Income|Gusto Benefits,2020-11-30,-1172.35,24754,2020-12-13 09:13:00
3,112,4522,82,Income|Interest,2020-11-30,4.24,24754,2020-12-13 09:13:00
4,113,4522,87,Other Income|Payroll,2020-11-30,-105115.57,24754,2020-12-13 09:13:00


In [123]:
cols = ['Main', 'Sub1', 'Sub2']
profit_df[cols] = profit_df['account_name'].str.split("|", expand=True)

cols = ['Main', 'Sub1', 'Sub2', 'Sub3', 'Sub5']
balance_df[cols] = balance_df['account_name'].str.split("|", expand=True)

In [124]:
profit_df['Sub3'] = np.nan
profit_df['Sub5'] = np.nan

profit_and_balance_df = pd.concat([profit_df, balance_df])

In [125]:
def net_profit_margin(df):
  revenue = pd.to_numeric(df[df['Sub1'] == 'Revenue']['balance']).mean()
  expenses = pd.to_numeric(df[df['Main'] == 'Expenses']['balance']).mean()
  if revenue == 0:
    return 0
  return (revenue - expenses) / (revenue)

In [126]:
def current_ratio(df) :
    current_assets = pd.to_numeric(df[df['Sub1'] == 'Current Assets']['balance']).mean()
    current_liabilities = pd.to_numeric(df[df['Sub1'] == 'Current Liabilities']['balance']).mean()
    if current_assets == 0:
        return 0
    return current_assets / current_liabilities

In [127]:
def working_capital(df):
    current_assets = pd.to_numeric(df[df['Sub1'] == 'Current Assets']['balance']).sum()
    current_liabilities = pd.to_numeric(df[df['Sub1'] == 'Current Liabilities']['balance']).sum()
    return current_assets - current_liabilities

In [128]:
def return_on_assets(df) :
    net_income = pd.to_numeric(df[df['Main'] == 'Income']['balance']).sum() + pd.to_numeric(df[df['Main'] == 'Other Income']['balance']).sum()
    current_assets = pd.to_numeric(df[df['Sub1'] == 'Current Assets']['balance']).sum()
    if current_assets == 0:
        return 0
    return net_income / current_assets

In [138]:
def get_revenue(df):
  revenue = pd.to_numeric(df[df['Sub1'] == 'Revenue']['balance']).sum()
  return revenue

In [129]:
out = pd.DataFrame()

In [139]:
pd_and_bd_grouping = profit_and_balance_df.groupby('period_date')

out['Return-On-Assets'] = pd_and_bd_grouping.apply(lambda x: return_on_assets(x))
out['Net-Profit-Margin'] = pd_and_bd_grouping.apply(lambda x: net_profit_margin(x))
out['Current-Ratio'] = pd_and_bd_grouping.apply(lambda x: current_ratio(x))
out['Working-Capital'] = pd_and_bd_grouping.apply(lambda x: working_capital(x))
out['Revenue'] = pd_and_bd_grouping.apply(lambda x: get_revenue(x))
out['Revenue-Volatility'] = out['Revenue'].pct_change()

In [140]:
out

,Return-On-Assets,Net-Profit-Margin,Current-Ratio,Working-Capital,Revenue,Revenue-Volatility
period_date,,,,,,
2012-11-30,0.000000,0.0,0.000000,0.000000,0.00,NaN
2012-12-31,0.000000,0.0,0.000000,0.000000,0.00,NaN
2013-01-31,0.000000,0.0,0.000000,0.000000,0.00,NaN
2013-02-28,0.000000,0.0,0.000000,0.000000,0.00,NaN
2013-03-31,0.000000,0.0,0.000000,0.000000,0.00,NaN
...,...,...,...,...,...,...
2020-07-31,0.503287,NaN,1.770979,170129.242667,420935.77,0.128376
2020-08-31,0.361435,NaN,2.025511,227393.263333,355663.75,-0.155064
2020-09-30,0.359553,NaN,2.833540,327430.930000,395077.41,0.110817


In [ ]:
def create_feature_dataframe(profit_and_balance_data):
    pd_and_bd_grouping = profit_and_balance_df.groupby('period_date')

    out = pd.DataFrame()
    out['Return-On-Assets'] = pd_and_bd_grouping.apply(lambda x: return_on_assets(x))
    out['Net-Profit-Margin'] = pd_and_bd_grouping.apply(lambda x: net_profit_margin(x))
    out['Current-Ratio'] = pd_and_bd_grouping.apply(lambda x: current_ratio(x))
    out['Working-Capital'] = pd_and_bd_grouping.apply(lambda x: working_capital(x))
    out['Revenue'] = pd_and_bd_grouping.apply(lambda x: get_revenue(x))
    out['Revenue-Volatility'] = out['Revenue'].pct_change()

    return out